In [ ]:
# =============================================================================
# All‑in‑one: install, clone converter, unzip, split, convert, register, train & eval
# =============================================================================

# 0) Install Detectron2 & basic PanopticAPI utilities
!pip install -q detectron2 -U
!pip install -q git+https://github.com/cocodataset/panopticapi.git

import os, sys, json, random, zipfile, shutil
from pathlib import Path

import numpy as np
import torch, cv2
from detectron2.data.datasets.coco_panoptic import register_coco_panoptic
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOPanopticEvaluator, inference_on_dataset
from detectron2 import model_zoo
from detectron2.config import get_cfg

torch.backends.cudnn.benchmark = True  # speed boost on GPU

# =============================================================================
# 1) Clone only the PanopticAPI converters folder (if not present)
# =============================================================================
conv_dir = Path.cwd() / "panopticapi_converters"
if not conv_dir.exists():
    print("Cloning PanopticAPI converters…")
    !git clone --depth=1 https://github.com/cocodataset/panopticapi.git panopticapi_src
    (Path.cwd() / "panopticapi_src" / "converters").rename(conv_dir)
    shutil.rmtree("panopticapi_src", ignore_errors=True)

# 2) Make that folder importable
sys.path.insert(0, str(conv_dir))
from detection2panoptic_coco_format import convert_detection_to_panoptic_coco_format
from panopticapi.utils import IdGenerator

# =============================================================================
# 3) Paths & parameters (idempotent)
# =============================================================================
zip_path    = Path("/home/manulab/Downloads/coco.zip.zip")  # your zip
nb_dir      = Path("/home/manulab/projects/computer vision portfolio/seg")
work_dir    = nb_dir / "coco_panoptic"
split_ratio = 0.8  # 80/20 train/val split

work_dir.mkdir(parents=True, exist_ok=True)

# =============================================================================
# 4) Unzip (idempotent)
# =============================================================================
flag = work_dir / ".unzipped"
if not flag.exists():
    print("→ Unzipping coco.zip …")
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(work_dir)
    flag.touch()
else:
    print("→ Already unzipped")

# =============================================================================
# 5) Point to your **fixed** layout
# =============================================================================
images_dir = Path("/home/manulab/projects/computer vision portfolio/seg/coco_panoptic/images")
cvat_ann   = Path("/home/manulab/projects/computer vision portfolio/seg/coco_panoptic/annotations/instances_default.json")


# =============================================================================
# 6) Build panoptic masks (idempotent)
# =============================================================================
pan_train  = work_dir / "panoptic_train2017"
pan_val    = work_dir / "panoptic_val2017"
train_imgs = work_dir / "train2017"
val_imgs   = work_dir / "val2017"

if not (anno_out / "panoptic_train2017.json").exists():
    print("→ Converting to COCO‑Panoptic …")
    img_ids   = [im["id"] for im in coco["images"]]
    random.seed(42)
    random.shuffle(img_ids)
    train_ids = set(img_ids[:int(len(img_ids)*split_ratio)])
    val_ids   = set(img_ids[int(len(img_ids)*split_ratio):])

    # Copy images into train/val folders
    train_imgs.mkdir(exist_ok=True)
    val_imgs.mkdir(exist_ok=True)
    for im in coco["images"]:
        src     = images_dir / im["file_name"].split('/')[-1]
        dst_dir = train_imgs if im["id"] in train_ids else val_imgs
        # Check if the source file exists before copying
        if not src.exists():
            print(f"Warning: Source file not found: {src}. Skipping this file.")
            continue
        shutil.copy2(src, dst_dir / im["file_name"].split('/')[-1])

    # Run panoptic conversion
    for split, idset, pano_dir, jn in [
        ("train", train_ids, pan_train, "panoptic_train2017.json"),
        ("val",   val_ids,   pan_val,   "panoptic_val2017.json")
    ]:
        pano_dir.mkdir(exist_ok=True)
        # Ensure 'images' subdirectory exists
        (pano_dir / 'images').mkdir(parents=True, exist_ok=True)
        out_json = anno_out / jn
        print(f"  • {split}: {len(idset)} images → {out_json.name}")
        convert_detection_to_panoptic_coco_format(
            input_json_file      = str(cvat_ann),
            segmentations_folder = str(pano_dir),
            output_json_file     = str(out_json),
            categories_json_file = str(conv_dir / "panoptic_coco_categories.json")
        )
else:
    print("→ Panoptic JSON & masks already present – skipping")
# =============================================================================
# 7) Register with Detectron2 & train
# =============================================================================
def register_split(name, img_dir, pan_dir, json_f):
    if name in DatasetCatalog.list():
        return name
    register_coco_panoptic(
        name,
        metadata={},
        image_root=str(img_dir),
        panoptic_root=str(pan_dir),
        panoptic_json=str(json_f),
        instances_json=""
    )
    return name

train_name = register_split(
    "cvat_pan_train",
    train_imgs,
    pan_train,
    anno_out / "panoptic_train2017.json"
)
val_name = register_split(
    "cvat_pan_val",
    val_imgs,
    pan_val,
    anno_out / "panoptic_val2017.json"
)
print("Datasets:", train_name, val_name)

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"
))
cfg.DATASETS.TRAIN = (train_name,)
cfg.DATASETS.TEST  = (val_name,)
cfg.SOLVER.IMS_PER_BATCH     = 4
cfg.SOLVER.BASE_LR           = 1e-3
cfg.SOLVER.MAX_ITER          = 10_000
cfg.SOLVER.STEPS             = (6_000, 8_000)
cfg.SOLVER.CHECKPOINT_PERIOD = 1_000
cfg.TEST.EVAL_PERIOD         = 1_000
cfg.DATALOADER.NUM_WORKERS   = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"
)
cfg.OUTPUT_DIR = str(nb_dir / "output_panoptic_fpn")
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(False)
trainer.train()

# =============================================================================
# 8) Evaluate
# =============================================================================
evaluator  = COCOPanopticEvaluator(val_name, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, val_name)
metrics    = inference_on_dataset(trainer.model, val_loader, evaluator)
print("Panoptic metrics:\n", metrics)


→ Already unzipped
→ Converting to COCO‑Panoptic …
  • train: 113 images → panoptic_train2017.json
CONVERTING...
COCO detection format:
	JSON file: /home/manulab/projects/computer vision portfolio/seg/coco_panoptic/annotations/instances_default.json
TO
COCO panoptic format
	Segmentation folder: /home/manulab/projects/computer vision portfolio/seg/coco_panoptic/panoptic_train2017
	JSON file: /home/manulab/projects/computer vision portfolio/seg/coco_panoptic/annotations/panoptic_train2017.json


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Number of cores: 32, images per core: 5
Core: 0, 0 from 5 images processed
Core: 1, 0 from 5 images processed
Core: 2, 0 from 5 images processed
Core: 3, 0 from 5 images processed
Core: 4, 0 from 5 images processed
Core: 6, 0 from 5 images processed
Core: 5, 0 from 5 images processed
Core: 7, 0 from 5 images processed
Core: 8, 0 from 5 images processedCore: 9, 0 from 5 images processed

Core: 10, 0 from 5 images pr

Traceback (most recent call last):
  File "/home/manulab/.local/lib/python3.10/site-packages/panopticapi/utils.py", line 16, in wrapper
    return f(*args, **kwargs)
  File "/home/manulab/projects/computer vision portfolio/seg/panopticapi_converters/detection2panoptic_coco_format.py", line 69, in convert_detection_to_panoptic_coco_format_single_core
    raise Exception("Segments for image {} overlap each other.".format(img_id))
Exception: Segments for image 16 overlap each other.


Caught exception in worker thread:Caught exception in worker thread:



Traceback (most recent call last):
  File "/home/manulab/.local/lib/python3.10/site-packages/panopticapi/utils.py", line 16, in wrapper
    return f(*args, **kwargs)
  File "/home/manulab/projects/computer vision portfolio/seg/panopticapi_converters/detection2panoptic_coco_format.py", line 69, in convert_detection_to_panoptic_coco_format_single_core
    raise Exception("Segments for image {} overlap each other.".format(img_id))
Exception: Segments for image 6 overlap each other.


Caught exception in worker thread:

...

Caught exception in worker thread:


Traceback (most recent call last):
  File "/home/manulab/.local/lib/python3.10/site-packages/panopticapi/utils.py", line 16, in wrapper
    return f(*args, **kwargs)
  File "/home/manulab/projects/computer vision portfolio/seg/panopticapi_converters/detection2panoptic_coco_format.py", line 69, in convert_detection_to_panoptic_coco_format_single_core
    raise Exception("Segments for image {} overlap each other.".format(img_id))
Exception: Segments for image 34 overlap each other.


Caught exception in worker thread:


Traceback (most recent call last):
  File "/home/manulab/.local/lib/python3.10/site-packages/panopticapi/utils.py", line 16, in wrapper
    return f(*args, **kwargs)
  File "/home/manulab/projects/computer vision portfolio/seg/panopticapi_converters/detection2panoptic_coco_format.py", line 69, in convert_detection_to_panoptic_coco_format_single_core
    raise Exception("Segments for image {} overlap each other.".format(img_id))
Exception: Segments for image 15 overlap each other.


Core: 26, all 4 images processed
